In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import argparse
from ss_baselines.av_nav.config import get_config
from ss_baselines.common.env_utils import construct_envs
from ss_baselines.common.environments import get_env_class


config = get_config(
    # config_paths="ss_baselines/av_nav/config/audionav/mp3d/env_test_0.yaml", # RGB + AudiogoalSensor
    config_paths="env_configs/audiogoal_rgb_depth_nocont.yaml",
    # opts=["CONTINUOUS", "False"],
    run_type="eval")
config.defrost()
config.NUM_PROCESSES = 1
config.USE_SYNC_VECENV = True
config.USE_VECENV = False
config.freeze()
# print(config)

envs = construct_envs(config, get_env_class(config.ENV_NAME))

/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-11 15:17:55,210 Initializing dataset AudioNav
2023-05-11 15:17:55,220 Initializing dataset AudioNav
2023-05-11 15:19:01,032 initializing sim SoundSpacesSim
2023-05-11 15:19:01,285 Initializing task AudioNav


In [2]:
obs = envs.reset()

In [3]:
envs.__dict__

{'_is_waiting': False,
 '_is_closed': True,
 '_num_envs': 1,
 '_auto_reset_done': True,
 'workers': [<ss_baselines.common.sync_vector_env.WorkerEnv at 0x7f97a1ed5850>],
 'observation_spaces': [Dict(audiogoal:Box(-3.4028235e+38, 3.4028235e+38, (2, 16000), float32), depth:Box(0.0, 1.0, (128, 128, 1), float32), rgb:Box(0, 255, (128, 128, 3), uint8), spectrogram:Box(-3.4028235e+38, 3.4028235e+38, (65, 26, 2), float32))],
 'action_spaces': [ActionSpace(MOVE_FORWARD:EmptySpace(), STOP:EmptySpace(), TURN_LEFT:EmptySpace(), TURN_RIGHT:EmptySpace())],
 '_paused': []}

In [4]:
envs.workers[0].__dict__

{'_env': <ss_baselines.common.environments.AudioNavRLEnv at 0x7f96066f9a30>,
 '_auto_reset_done': True}

In [5]:
envs.workers[0]._env.__dict__

{'_rl_config': Config({'SUCCESS_REWARD': 10.0, 'SLACK_REWARD': -0.01, 'WITH_TIME_PENALTY': True, 'WITH_DISTANCE_REWARD': True, 'DISTANCE_REWARD_SCALE': 1.0, 'TIME_DIFF': False, 'PPO': Config({'clip_param': 0.1, 'ppo_epoch': 4, 'num_mini_batch': 1, 'value_loss_coef': 0.5, 'entropy_coef': 0.2, 'lr': 0.00025, 'eps': 1e-05, 'max_grad_norm': 0.5, 'num_steps': 150, 'hidden_size': 512, 'use_gae': True, 'use_linear_lr_decay': True, 'use_linear_clip_decay': True, 'gamma': 0.99, 'tau': 0.95, 'reward_window_size': 50})}),
 '_core_env_config': Config({'SEED': 100, 'ENVIRONMENT': Config({'MAX_EPISODE_STEPS': 500, 'MAX_EPISODE_SECONDS': 10000000, 'ITERATOR_OPTIONS': Config({'CYCLE': True, 'SHUFFLE': True, 'GROUP_BY_SCENE': True, 'NUM_EPISODE_SAMPLE': -1, 'MAX_SCENE_REPEAT_EPISODES': -1, 'MAX_SCENE_REPEAT_STEPS': 10000, 'STEP_REPETITION_RANGE': 0.2})}), 'TASK': Config({'REWARD_MEASURE': 'distance_to_goal', 'SUCCESS_MEASURE': 'spl', 'SUCCESS_REWARD': 2.5, 'SLACK_REWARD': -0.01, 'END_ON_SUCCESS': False

In [6]:
envs.workers[0]._env._env.__dict__

{'_config': Config({'SEED': 100, 'ENVIRONMENT': Config({'MAX_EPISODE_STEPS': 500, 'MAX_EPISODE_SECONDS': 10000000, 'ITERATOR_OPTIONS': Config({'CYCLE': True, 'SHUFFLE': True, 'GROUP_BY_SCENE': True, 'NUM_EPISODE_SAMPLE': -1, 'MAX_SCENE_REPEAT_EPISODES': -1, 'MAX_SCENE_REPEAT_STEPS': 10000, 'STEP_REPETITION_RANGE': 0.2})}), 'TASK': Config({'REWARD_MEASURE': 'distance_to_goal', 'SUCCESS_MEASURE': 'spl', 'SUCCESS_REWARD': 2.5, 'SLACK_REWARD': -0.01, 'END_ON_SUCCESS': False, 'TYPE': 'AudioNav', 'SENSORS': ['AUDIOGOAL_SENSOR', 'SPECTROGRAM_SENSOR'], 'MEASUREMENTS': ['DISTANCE_TO_GOAL', 'NORMALIZED_DISTANCE_TO_GOAL', 'SUCCESS', 'SPL', 'SOFT_SPL', 'NUM_ACTION', 'SUCCESS_WEIGHTED_BY_NUM_ACTION'], 'GOAL_SENSOR_UUID': 'spectrogram', 'POSSIBLE_ACTIONS': ['STOP', 'MOVE_FORWARD', 'TURN_LEFT', 'TURN_RIGHT'], 'MAX_COLLISIONS': -1.0, 'COUNT_OBJ_COLLISIONS': True, 'COUNT_ROBOT_OBJ_COLLS': False, 'SETTLE_STEPS': 5, 'CONSTRAINT_VIOLATION_ENDS_EPISODE': True, 'FORCE_REGENERATE': False, 'SHOULD_SAVE_TO_CAC

In [11]:
envs.workers[0]._env._env._sim._oracle_actions

[1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 0]

In [15]:
envs.reset()
envs.workers[0]._env._env._sim._oracle_actions

[1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 3, 1, 0]

## Attempt at using the `ShortestPathFollower` class

In [7]:
envs.workers[0]._env._env._sim.__dict__

{'config': Config({'TYPE': 'SoundSpacesSim', 'ACTION_SPACE_CONFIG': 'v0', 'FORWARD_STEP_SIZE': 1.0, 'CREATE_RENDERER': False, 'REQUIRES_TEXTURES': True, 'LAG_OBSERVATIONS': 0, 'AUTO_SLEEP': False, 'STEP_PHYSICS': True, 'UPDATE_ROBOT': True, 'CONCUR_RENDER': False, 'NEEDS_MARKERS': True, 'SCENE': 'data/scene_datasets/mp3d/PuKPg4mmafe/PuKPg4mmafe.glb', 'SCENE_DATASET': 'default', 'ADDITIONAL_OBJECT_PATHS': [], 'SEED': 100, 'TURN_ANGLE': 10, 'TILT_ANGLE': 15, 'DEFAULT_AGENT_ID': 0, 'DEBUG_RENDER': False, 'DEBUG_RENDER_GOAL': True, 'ROBOT_JOINT_START_NOISE': 0.0, 'ARM_REST': [0.6, 0.0, 0.9], 'CTRL_FREQ': 120.0, 'AC_FREQ_RATIO': 4, 'ROBOT_URDF': 'data/robots/hab_fetch/robots/hab_fetch.urdf', 'ROBOT_TYPE': 'FetchRobot', 'EE_LINK_NAME': None, 'LOAD_OBJS': False, 'HOLD_THRESH': 0.09, 'GRASP_IMPULSE': 1000.0, 'IK_ARM_URDF': 'data/robots/hab_fetch/robots/fetch_onlyarm.urdf', 'RGB_SENSOR': Config({'HEIGHT': 128, 'WIDTH': 128, 'POSITION': [0, 1.25, 0], 'ORIENTATION': [0.0, 0.0, 0.0], 'HFOV': 90, '

In [10]:
envs.workers[0]._env._env._sim._sim # <soundspaces.simulator.DummySimulator at 0x7f9556ccf910>

In [9]:
import habitat_sim
from habitat.sims.habitat_simulator.actions import HabitatSimActions
from habitat.sims.habitat_simulator.habitat_simulator import HabitatSim

type(envs.workers[0]._env._env._sim)

envs.workers[0]._env._env._sim.make_greedy_follower(
    agent_id = 0,
    goal_radius = 0.5,
    stop_key=HabitatSimActions.STOP,
    forward_key=HabitatSimActions.MOVE_FORWARD,
    left_key=HabitatSimActions.TURN_LEFT,
    right_key=HabitatSimActions.TURN_RIGHT
)

AttributeError: 'DummySimulator' object has no attribute 'make_greedy_follower'

In [ ]:
# Instantiate a Shortest Path Follower agent
from soundspaces.tasks.shortest_path_follower import ShortestPathFollower

spf_agent = ShortestPathFollower(sim = envs.workers[0]._env._env._sim, goal_radius = 0.5)
spf_agent._build_follower()

### Attempt at extracting Top Down Map and other trajectory analysis information

In [ ]:
action_fwd = {"action": "MOVE_FORWARD", "action_args": None}
observation, reward, done, info = envs.step(**action_fwd)

In [ ]:
info

In [ ]:
info["top_down_map"]["map"].shape, info["top_down_map"]["fog_of_war_mask"].shape

In [ ]:
plt.imshow(info["top_down_map"]["map"])

In [ ]:
plt.imshow(info["top_down_map"]["fog_of_war_mask"])

In [ ]:
# (info["top_down_map"]["map"] * info["top_down_map"]["fog_of_war_mask"]).shape
plt.imshow(info["top_down_map"]["map"] * info["top_down_map"]["fog_of_war_mask"])